In [1]:
import getpass
# And pyspark.sql to get the spark session
from pathlib import Path
import pickle
import os
import pandas as pd
import pickle
import os
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm 
import matplotlib.pyplot as plt 
import annoy

In [2]:
user_vec = pd.read_parquet("user_vec_small.parquet",engine = 'pyarrow')

In [3]:
item_vec =  pd.read_parquet("item_vec_small.parquet",engine = 'pyarrow')

In [4]:
pwd

'/home/rz2432/final-project-group_101/CheckPoint Submission/fast_search'

In [5]:
ratings_full_train = pd.read_csv('/home/rz2432/final-project-group_101/CheckPoint Submission/ratings_small_train.csv', names = ('userId', 'movieId', 'rating', 'timestamp'))
ratings_full_val = pd.read_csv('/home/rz2432/final-project-group_101/CheckPoint Submission/ratings_small_val.csv', names = ('userId', 'movieId', 'rating', 'timestamp', "median_timestamp"))
ratings_full_test = pd.read_csv('/home/rz2432/final-project-group_101/CheckPoint Submission/ratings_small_test.csv', names = ('userId', 'movieId', 'rating', 'timestamp', "median_timestamp"))

ratings_full_val = ratings_full_val.drop(columns=['median_timestamp'])
ratings_full_test = ratings_full_test.drop(columns=['median_timestamp'])
ratings_full_train = ratings_full_train.drop(columns=['timestamp'])
ratings_full_val = ratings_full_val.drop(columns=['timestamp'])
ratings_full_test = ratings_full_test.drop(columns=['timestamp'])

In [6]:
total_item_user = pd.concat([ratings_full_train, ratings_full_test, ratings_full_val]).drop_duplicates()
#sort by ratings
total_item_user = total_item_user.sort_values(['rating'])
total_item_user['new_movieId'] = (total_item_user.groupby(['movieId'], sort=False).ngroup()+1)

In [7]:
label = total_item_user.groupby('userId')['movieId'].apply(list).to_numpy()

In [8]:
label[1]

[114060,
 91658,
 77455,
 318,
 71535,
 109487,
 91529,
 99114,
 115713,
 8798,
 3578,
 6874,
 74458,
 79132,
 112552,
 333,
 46970,
 48516,
 86345,
 80489,
 68157,
 58559,
 1704,
 80906,
 60756,
 122882,
 89774,
 106782,
 131724]

In [9]:
user = user_vec['features'].to_numpy()
item = item_vec['features'].to_numpy()

In [10]:
item = np.array([x for x in item]) 
user = np.array([x for x in user]) 

Fast search: use a spatial data structure (e.g., LSH or partition trees) to implement accelerated search at query time. For this, it is best to use an existing library such as annoy, nmslib, or scann and you will need to export the model parameters from Spark to work in your chosen environment. For full credit, you should provide a thorough evaluation of the efficiency gains provided by your spatial data structure over a brute-force search method.

In [11]:
true_label = np.array([x for x in label]) 

<ipython-input-11-2127fb04794e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  true_label = np.array([x for x in label])


In [12]:
def query_one_user(item,user,query,k):
    start = time.time()
    res = np.argsort(-item.dot(query))[:k]
    time_taken = time.time() - start
    return res,time_taken

In [13]:
query_one_user(item,user,user[600],10)

(array([5677, 4270, 2920, 6986,  344, 2707,    8, 2330,  886, 6507]),
 0.0007522106170654297)

In [18]:
def precision(pred, label):
    count = 0
    for i in range(len(pred)):
        if pred[i] in label:
            count +=1
    return count/len(pred)

In [23]:
def brutal_search(items,users,k,label):
    n_items = item.shape[0]
    n_feat = item.shape[1]
    n_users = user.shape[0]
    
    time_list = []
    result_list = []
    acc = []
    
    for i in tqdm(range(n_users)):
        neighbours,time_taken = query_one_user(items,users,users[i],k)
        prec = precision(neighbours, label[i])
        time_list.append(time_taken) 
        result_list.append(neighbours)
        acc.append(prec)
        

    avg_time = sum(time_list) / len(time_list) 
    accu = sum(acc) / len(acc) 
    print(avg_time)
    print('brute search time:%.6f' % avg_time)
    print('accuracy:%.6f' % accu)
    
    return avg_time,result_list

In [24]:
time_b,result_b = brutal_search(item,user,100,true_label)

<ipython-input-23-30bf195d9ed0>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/610 [00:00<?, ?it/s]

0.0005664954419995917
brute search time:0.000566
accuracy:0.016393


In [ ]:
for i in 

### Fast search with annoy

In [25]:
def build_tree(item,n_trees):
    f = len(item[0])
    t = annoy.AnnoyIndex(f, metric='dot') 
    for i in range(len(item)):
        t.add_item(i, item[i])
    t.build(n_trees)
    return t

In [26]:
def wrapup(obj, method, mapping): 
    get_map = lambda x: [x[mapping[i]] for i in range(len(mapping))]
    def wrapped(*args, **kwrds):
        return obj.__getattribute__(method)(*get_map(args)) 
    return wrapped

In [27]:
def fast_query_one_user(item,user,query,k,param,model_wrapped):
    start = time.time()
    res = model_wrapped(query, k, 100) 
    interval = time.time() - start
    return interval, res

In [28]:
tree = build_tree(item,100)
wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
time_taken,result = fast_query_one_user(item,user,user[0],10,[0,1,2],wrapped)
print(time_taken,result)

0.00014662742614746094 [6839, 1211, 75, 795, 3930, 1099, 6378, 5459, 6575, 2058]


In [29]:
def precision(pred, label):
    count = 0
    for i in range(len(pred)):
        if pred[i] in label:
            count +=1
    return count/len(pred)

In [30]:
precision(result, true_label[0])

0.1

In [31]:
def fast_search_with_annoy(item,user,label,model_wrapped,k):
    n_items = item.shape[0]
    n_feat = item.shape[1]
    n_users = user.shape[0]
    
    times = []
    precisions = []
    results = []

    for i in tqdm(range(n_users)):
        interval, res = fast_query_one_user(item,user,user[i],k,[0,1,2],model_wrapped)
        prec = precision(res, label[i]) 
        results.append(res)
        times.append(interval) 
        precisions.append(prec)

    mean_time = sum(times) / len(times) 
    mean_precision = sum(precisions) / len(precisions)
    print('-' * 26)
    print('Mean Query Search Time: %.6f' % mean_time) 
    print('Mean precision: %.6f' % mean_precision)
    return mean_time, mean_precision,results

In [32]:
tree = build_tree(item,100)
wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
proces_time, accuracy, result = fast_search_with_annoy(item,user,true_label, wrapped, 100) 

<ipython-input-31-8c63e029819a>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/610 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000053
Mean precision: 0.019754


In [33]:
n_trees = [10,100,200,500,700,1000]

loop_time = []
loop_precision = []
loop_res = []
selection = []
for k in n_trees:
    tree = build_tree(item,k)
    wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
    proces_time, accuracy, result = fast_search_with_annoy(item,user,true_label, wrapped, 100)
    selection.append(k)
    loop_time.append(proces_time)
    loop_precision.append(accuracy)
    loop_res.append(result)

<ipython-input-31-8c63e029819a>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/610 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000027
Mean precision: 0.016656


  0%|          | 0/610 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000051
Mean precision: 0.019754


...

--------------------------
Mean Query Search Time: 0.000151
Mean precision: 0.013574


  0%|          | 0/610 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000210
Mean precision: 0.013557


  0%|          | 0/610 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000330
Mean precision: 0.013607


In [34]:
evaluation = {"params":selection,"times":loop_time,"accuracy":loop_precision}

In [35]:
pd.DataFrame(evaluation)

,params,times,accuracy
0,10,0.000027,0.016656
1,100,0.000051,0.019754
2,200,0.000087,0.019082
3,500,0.000151,0.013574
4,700,0.000210,0.013557
5,1000,0.000330,0.013607


In [36]:
#tree = 100
tree = build_tree(item,100)
wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
proces_time, accuracy, result = fast_search_with_annoy(item,user,true_label, wrapped, 100) 

<ipython-input-31-8c63e029819a>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/610 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000057
Mean precision: 0.019754
